### Imports

In [149]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm

In [150]:
url = r"https://www.goodreads.com/author/quotes/656983.J_R_R_Tolkien?page=1"

driver = webdriver.Chrome()
driver.get(url)

In [151]:
all_quotes = []

In [152]:
quotes_per_page = driver.find_elements(By.CLASS_NAME, "quoteText")
quotes_per_page = [quote.text for quote in quotes_per_page]
all_quotes.extend(quotes_per_page)
quotes_per_page.clear()
print(all_quotes)

['“All that is gold does not glitter,\nNot all those who wander are lost;\nThe old that is strong does not wither,\nDeep roots are not reached by the frost.\n\nFrom the ashes a fire shall be woken,\nA light from the shadows shall spring;\nRenewed shall be blade that was broken,\nThe crownless again shall be king.”\n― J.R.R. Tolkien, The Fellowship of the Ring', '“Not all those who wander are lost.”\n― J.R.R. Tolkien, The Fellowship of the Ring', '“I wish it need not have happened in my time," said Frodo.\n"So do I," said Gandalf, "and so do all who live to see such times. But that is not for them to decide. All we have to decide is what to do with the time that is given us.”\n― J.R.R. Tolkien, The Fellowship of the Ring', "“I don't know half of you half as well as I should like; and I like less than half of you half as well as you deserve.”\n― J.R.R. Tolkien, The Fellowship of the Ring", '“All we have to decide is what to do with the time that is given us.”\n― J.R.R. Tolkien, The Fello

In [153]:
next_page_button = driver.find_element(By.CLASS_NAME, "next_page").click()

In [154]:
from selenium.common import TimeoutException

try:
    time.sleep(2)
    login_popup = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "gr-h3 gr-h3--serif gr-h3--noMargin"))
    )
    login_popup.send_keys(Keys.ESCAPE)

except TimeoutException as e:
    print("Error occurred:", e.msg)

Error occurred: 
